Produce spectrum data ***energies.dat*** & ***oscst.dat*** 

In [ ]:
# ----------------------------- Gen spectrum data [energies.dat] & [oscst.dat] --------------------------
def genEnergyAndOscil(property_dir="GaussOutput"):
    '''
      take in file (Output) 
      find matching geom (Input) 
      extract prop from Output, grab geom from Input 
      write to xyz output 1. Energy & F, 2. geom
    '''
    # -------------------- open file to write --------------------
    fileWriteName = os.path.join(".", "spectrumData","energies.dat")
    os.makedirs(os.path.dirname(fileWriteName), exist_ok=True) # create folder if not existed yet
    fileWriter = open(fileWriteName,"w+")

    fileWriteName2 = os.path.join(".", "spectrumData","oscst.dat")
    os.makedirs(os.path.dirname(fileWriteName2), exist_ok=True) # create folder if not existed yet
    fileWriter2 = open(fileWriteName2,"w+")

    # ----------------------------- scan for matching geoms in/output files --------------------------
    for f in os.listdir(property_dir): # loop output
        filePath1 = os.path.join(".",property_dir,f)
        # ------------------- extract prop from f, geom from f2 -----------------
        # read prop over
        prop = extract_prop(filePath1)
        delim = ", "
        print(str(prop))
        energy1, os1 = prop[0]
        energy2, os2 = prop[1]
        # energy2, os2 = prop[1]
        # res = delim.join(map(str, prop[state]))
        fileWriter.write(str(energy1)+" "+str(energy2)+"\n")
        fileWriter2.write(str(os1)+" "+str(os2)+"\n")
    
    fileWriter.close()
    fileWriter2.close()

genEnergyAndOscil() # build for each state 0 / 1

From here send  ***energies.dat*** & ***oscst.dat*** to Gaussian to run ***abs-spec-config.f90*** 

In [ ]:
........

Extract spectrum data zip & Remove non-data files

In [ ]:
!unzip ./ABS_CONFIG_0-10.zip

In [ ]:
# ---------------------- script for drawing spectrum ---------------------- 
mydir = "ABS_CONFIG_0-02"
for f in os.listdir(mydir):
    if not f.endswith(".dat"):
        continue
    os.remove(os.path.join(mydir, f))

Normalize spectrum data

In [ ]:
# ^ *(\d+.\d+) *(\d+.\d+)
import pandas as pd
import os
import numpy as np
abs_folder = "ABS_CONFIG_0-02"
final_df = None
for fileN in os.listdir(abs_folder):
    df = pd.read_csv(os.path.join(abs_folder,fileN), header = None, sep ='\s+',engine = 'python')  
    #Convert to numeric
    for col in df:
        df[col] = pd.to_numeric(df[col])
    if final_df is None:
        final_df = df.copy()
    else:
        # final_df[0] = final_df[2]+df[2] 
        final_df[1] = final_df[1]+df[1] 

# Create an object to transform the data to fit minmax processor
# final_df[1] = min_max_scaler.fit_transform(final_df[1])
final_df[1]=(final_df[1]-final_df[1].min())/(final_df[1].max()-final_df[1].min())

# # Run the normalizer on the dataframe
# df_normalized = pd.DataFrame(y_scaled)
np.savetxt(os.path.join(".",abs_folder,"spectrum.txt"), final_df, fmt='%16.4e' , newline="\r\n")

Check results

In [ ]:
results = np.loadtxt(os.path.join(".",abs_folder,"spectrum.txt"))
results

Plot spectrum

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
results = np.loadtxt(os.path.join(".",abs_folder,"spectrum.txt"))
results2 = np.loadtxt(os.path.join(".",abs_folder,"spectrum_pred.txt"))

time = results[:,0]
train_loss = results[:,1]
time2 = results2[:,0]
train_loss2 = results2[:,1]


plt.figure(figsize=(15,8))
# plt.subplot(1,2,1)
# plt.plot(time, val_loss, label='Validation')
# plt.plot(time, train_loss, label='truth-value')
plt.plot(time2, train_loss2, label='predicted',c='darkorange')
# plt.yscale('log')
plt.ylabel('Abs')
plt.xlabel('Energy [eV]')
plt.xticks(np.arange(min(time), max(time), 0.51))
plt.legend()
plt.show()